<a href="https://colab.research.google.com/github/YanihGOAT/nula/blob/main/Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Intro au data




# Intro data

In [1]:
##Intro au data
!wget https://github.com/crux82/squad-it/raw/master/SQuAD_it-train.json.gz #importation dun dataset hors HGF
!wget https://github.com/crux82/squad-it/raw/master/SQuAD_it-test.json.gz
!gzip -dkv SQuAD_it-*.json.gz

--2025-11-21 05:01:03--  https://github.com/crux82/squad-it/raw/master/SQuAD_it-train.json.gz
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/crux82/squad-it/master/SQuAD_it-train.json.gz [following]
--2025-11-21 05:01:04--  https://raw.githubusercontent.com/crux82/squad-it/master/SQuAD_it-train.json.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7725286 (7.4M) [application/octet-stream]
Saving to: ‘SQuAD_it-train.json.gz’

SQuAD_it-train.json 100%[===================>]   7.37M  27.5MB/s    in 0.3s    

2025-11-21 05:01:05 (27.5 MB/s) - ‘SQuAD_it-train.json.gz’ saved [7725286/7725286]

--2

In [2]:
from datasets import load_dataset

squad_it_dataset = load_dataset("json", data_files="SQuAD_it-train.json", field="data")
squad_it_dataset

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['title', 'paragraphs'],
        num_rows: 442
    })
})

In [3]:
#Dans l'étape d'avant on a load seulement le set de train nous on veut celui de train + celui de test dans un seul data set

data_files = {"train": "SQuAD_it-train.json", "test": "SQuAD_it-test.json" }
squad_it_dataset = load_dataset("json", data_files=data_files, field="data")
squad_it_dataset

#peut également déziper directement donc pas besoin de décompresser le fichier et peut également prendre directement un url que ça soit sur le pc ou sur le web tu fais
#url = "https://github.com/crux82/squad-it/raw/master/"
#data_files = { "train": url + "SQuAD_it-train.json.gz","test": url + "SQuAD_it-test.json.gz",}


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['title', 'paragraphs'],
        num_rows: 442
    })
    test: Dataset({
        features: ['title', 'paragraphs'],
        num_rows: 48
    })
})

# Modifié et créer des dataset

In [21]:
!wget "https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip"
!unzip drugsCom_raw.zip
from datasets import load_dataset

data_files = {"train": "drugsComTrain_raw.tsv", "test": "drugsComTest_raw.tsv"}
# \t is the tab character in Python
drug_dataset = load_dataset("csv", data_files=data_files, delimiter="\t")



--2025-11-21 05:27:19--  https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘drugsCom_raw.zip’

drugsCom_raw.zip        [           <=>      ]  41.00M  15.4MB/s    in 2.7s    

2025-11-21 05:27:23 (15.4 MB/s) - ‘drugsCom_raw.zip’ saved [42989872]

Archive:  drugsCom_raw.zip
  inflating: drugsComTest_raw.tsv    
  inflating: drugsComTrain_raw.tsv   


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [22]:
drug_sample = drug_dataset["train"].shuffle(seed=42).select(range(1000))
drug_sample[:3]

{'Unnamed: 0': [87571, 178045, 80482],
 'drugName': ['Naproxen', 'Duloxetine', 'Mobic'],
 'condition': ['Gout, Acute', 'ibromyalgia', 'Inflammatory Conditions'],
 'review': ['"like the previous person mention, I&#039;m a strong believer of aleve, it works faster for my gout than the prescription meds I take. No more going to the doctor for refills.....Aleve works!"',
  '"I have taken Cymbalta for about a year and a half for fibromyalgia pain. It is great\r\nas a pain reducer and an anti-depressant, however, the side effects outweighed \r\nany benefit I got from it. I had trouble with restlessness, being tired constantly,\r\ndizziness, dry mouth, numbness and tingling in my feet, and horrible sweating. I am\r\nbeing weaned off of it now. Went from 60 mg to 30mg and now to 15 mg. I will be\r\noff completely in about a week. The fibro pain is coming back, but I would rather deal with it than the side effects."',
  '"I have been taking Mobic for over a year with no side effects other than 

In [33]:
#Dans ce data set on voi plusieurs petites erreurs qu'il faut corriger : condition à l'air d'avoir des majuscule
#random on veut uniformiser ça pour pas que y'ait de différence entre les même mots
#La colonne unamed :0 à l'air d'être un ID de patient donc mieux vaut la renommer
#y'a différent type de line separator et du code html on va aussi unormiser ça

#drug_dataset = drug_dataset.rename_column( original_column_name= "Unnamed: 0", new_column_name= "patient_id")
drug_dataset
drug_dataset = drug_dataset.filter(lambda x: x["condition"] is not None)

def lowercase_condition(example):
  return { "condition" : example["condition"].lower()}
drug_dataset=drug_dataset.map(lowercase_condition)

drug_dataset["train"]["condition"][:3]

Filter:   0%|          | 0/160398 [00:00<?, ? examples/s]

Filter:   0%|          | 0/53471 [00:00<?, ? examples/s]

Map:   0%|          | 0/160398 [00:00<?, ? examples/s]

Map:   0%|          | 0/53471 [00:00<?, ? examples/s]

['left ventricular dysfunction', 'adhd', 'birth control']

In [36]:
#on va rajouter dans le data set une colonne qui donne le nbr de mot dans une review
def compute_review_length(example):
    return {"review_length": len(example["review"].split())}
drug_dataset= drug_dataset.map(compute_review_length)

Map:   0%|          | 0/160398 [00:00<?, ? examples/s]

Map:   0%|          | 0/53471 [00:00<?, ? examples/s]

In [39]:
#drug_dataset["train"][0] y'a bien une nvl colonne

#mtn vu qu'on veut analyser la condition des patiens autant juste retirer les review pas longue car elles ne nous aident pas
drug_dataset = drug_dataset.filter(lambda x: x["review_length"] > 30 )
print(drug_dataset.num_rows)

drug_dataset = drug_dataset.map(lambda x: {"review" : html.unescape(x["review"])})

{'train': 160398, 'test': 53471}


Filter:   0%|          | 0/160398 [00:00<?, ? examples/s]

Filter:   0%|          | 0/53471 [00:00<?, ? examples/s]

{'train': 138514, 'test': 46108}
